In [183]:
import pandas as pd
import string
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [184]:
df = pd.read_csv('train.csv', index_col="Loan_ID")

In [185]:
df.head()

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
Loan_ID,,,,,,,,,,,,,
10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1


In [186]:
df.isna().sum()

Loan_Amount_Requested          0
Length_Employed             7371
Home_Owner                 25349
Annual_Income              25102
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    88379
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
Interest_Rate                  0
dtype: int64

#### Handling missing data

In [187]:
def cleaning_data(df):
    df["Loan_Amount_Requested"] = df["Loan_Amount_Requested"].str.replace(",","")
    df["Loan_Amount_Requested"] = df["Loan_Amount_Requested"].astype(float)
    df["Length_Employed"] = df["Length_Employed"].str.replace(r'[><+]+',"")
    df["Length_Employed"] = df["Length_Employed"].str.extract('(\d+)').astype(float)
    df.drop(df[df.Annual_Income.isna()].index, inplace=True)
    df["Length_Employed"] = df.groupby(["Annual_Income"])["Length_Employed"].transform(lambda x: x.fillna(x.mean()))
    df.drop(df[df.Length_Employed.isna()].index, inplace=True)
    #df["Home_Owner"] = df["Home_Owner"].transform(lambda x: x.fillna("Other"))
    return df

In [188]:
df = cleaning_data(df)

In [189]:
df.isna().sum()

Loan_Amount_Requested          0
Length_Employed                0
Home_Owner                 21199
Annual_Income                  0
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    74027
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
Interest_Rate                  0
dtype: int64

In [190]:
df["Home_Owner"] = df["Home_Owner"].astype("category")
## Try testing this column by using ordered category later on

In [191]:
df["Home_Owner"]

Loan_ID
10000001        Rent
10000003    Mortgage
10000004         NaN
10000005         Own
10000006        Rent
              ...   
10164299    Mortgage
10164300    Mortgage
10164301        Rent
10164303         NaN
10164304        Rent
Name: Home_Owner, Length: 137768, dtype: category
Categories (5, object): [Mortgage, None, Other, Own, Rent]

In [192]:
df.isna().sum()

Loan_Amount_Requested          0
Length_Employed                0
Home_Owner                 21199
Annual_Income                  0
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    74027
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
Interest_Rate                  0
dtype: int64

In [193]:
df["Loan_Amount_Requested"].describe()

count    137768.000000
mean      14382.778294
std        8285.674603
min         500.000000
25%        8000.000000
50%       12200.000000
75%       20000.000000
max       35000.000000
Name: Loan_Amount_Requested, dtype: float64

In [199]:
scaler = MinMaxScaler()
loan_amount = df["Loan_Amount_Requested"].values
loan_amount = loan_amount.reshape(1,-1)

In [200]:
scaler.fit(df["Loan_Amount_Requested"])
loan_amount = scaler.transform(loan_amount)
loan_amount = loan_amount.reshape(-1,1).tolist()

ValueError: Expected 2D array, got 1D array instead:
array=[ 7000. 24725. 16000. ... 24000. 24000. 35000.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [196]:
loan_amount = [val for sublist in loan_amount for val in sublist]
loan_amount = pd.Series(loan_amount)

In [197]:
df.replace(df["Loan_Amount_Requested"], loan_amount, inplace=True)

In [198]:
df

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
Loan_ID,,,,,,,,,,,,,
10000001,7000.0,1.0,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
10000003,24725.0,7.0,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
10000004,16000.0,1.0,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
10000005,17000.0,8.0,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1
10000006,4500.0,2.0,Rent,30000.0,VERIFIED - income source,credit_card,10.88,1,NaN,12,15,Male,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10164299,25000.0,1.0,Mortgage,70000.0,VERIFIED - income source,small_business,0.79,0,NaN,8,27,Male,2
10164300,3000.0,3.0,Mortgage,45000.0,VERIFIED - income source,major_purchase,8.43,2,NaN,25,36,Male,2
10164301,24000.0,1.0,Rent,108000.0,VERIFIED - income source,other,14.30,0,22.0,12,36,Female,3
